In [1]:
import numpy as np
import pandas as pd
import re

# Source data wranlging

## Reads

### Raw reads

In [2]:
to_replace = {"B_caccae_ATCC_43185":"B_caccae",
              "B_WH2_closed":"B_cellulosilyticus_WH2",
              "B_ovatus_ATCC_8483":"B_ovatus",
              "B_thetaiotaomicron_VPI-5482":"B_thetaiotaomicron",
              "B_uniformis_ATCC_8492":"B_uniformis",
              "B_vulgatus_ATCC_8482":"B_vulgatus",
              "C_aerofaciens_ATCC_25986":"C_aerofaciens",
              "C_scindens_ATCC_35704":"C_scindens",
              "C_spiroforme_DSM_1552":"C_spiroforme",
              "D_longicatena_DSM_13814":"D_longicatena",
              "P_distasonis_ATCC_8503":"P_distasonis",
              "R_obeum_ATCC_29174":"R_obeum"}
to_drop = ['14-28']
for i in range(1, 16):
    if len(str(i)) == 1:
        to_drop.append(f'0{i}-41')
    else:
        to_drop.append(f'{(i)}-41')
to_drop

['14-28',
 '01-41',
 '02-41',
 '03-41',
 '04-41',
 '05-41',
 '06-41',
 '07-41',
 '08-41',
 '09-41',
 '10-41',
 '11-41',
 '12-41',
 '13-41',
 '14-41',
 '15-41']

In [3]:
reads_raw = pd.read_excel('McNulty_data_source.xlsx', sheet_name='Raw_counts')
reads_raw

,name,01-01,01-01-r1,01-02,01-03,01-05,01-07,01-10,01-12,01-13,...,15-29,15-30,15-31,15-33,15-35,15-38,15-40,15-41,15-42,15-CEC
0,B_caccae_ATCC_43185,4662,28518,72598,90757,90761,43961,38713,47831,42591,...,27031,73923,65570,203081,204656,225795,261540,145981,250616,401542
1,B_ovatus_ATCC_8483,8330,66063,169916,157855,182563,69461,5558,77031,86669,...,17201,7579,1767,5301,6507,6843,8071,6086,15165,16946
2,B_thetaiotaomicron_VPI-5482,3639,24562,76142,147517,211588,82272,14337,54122,55977,...,23595,53246,42963,137392,118051,164264,204214,183137,252820,332110
3,B_uniformis_ATCC_8492,440,4428,4617,14112,32379,14689,3338,15498,15371,...,4612,11342,7656,22550,17774,27056,23763,18286,49599,53369
4,B_vulgatus_ATCC_8482,17711,412906,132224,104042,159878,77457,28659,36031,39085,...,53179,91247,44173,167332,200621,232121,328177,161095,268588,326432
5,B_WH2_closed,8878,61078,281681,436720,834727,312998,49724,229893,311218,...,55939,75706,57077,256189,224229,313596,246003,75657,88325,101017
6,C_aerofaciens_ATCC_25986,60,187,2844,5418,6777,3186,205,2419,2415,...,726,1062,711,347,668,536,358,579,2142,2115
7,C_scindens_ATCC_35704,1565,15886,11403,10153,11103,5237,1321,6789,4148,...,5187,6106,7428,6985,11135,7155,7066,16116,34857,39587
8,C_spiroforme_DSM_1552,1,76,1664,3604,1482,159,314,183,33,...,48,50,54,380,660,292,405,481,442,1407
9,D_longicatena_DSM_13814,38,775,108,7,1,2,0,1,1,...,7,10,10,20,29,30,49,14,20,47


In [4]:
reads_raw['name'].replace(to_replace, inplace=True)
reads_raw.sort_values(by='name', inplace=True)
reads_raw.set_index('name', inplace=True)
r1 = re.compile('\d\d-\d\d-r\d')
r1s = [col for col in reads_raw if r1.match(col)]
rs = [i.replace('-r1', '') for i in r1s]
for i,j in zip(r1s, rs):
    reads_raw[j] = reads_raw[i]
    reads_raw.drop(i, axis=1, inplace=True)
cec = re.compile('\d\d-CEC')
for col in reads_raw:
    if cec.match(col):
        reads_raw.drop(col, axis=1, inplace=True)
col_names = []
for col in reads_raw.columns:
    mouse, day = col.split('-')
    day = str(int(day) - 1)
    if len(day) == 1:
        day = '0' + day
    col_names.append(f'{mouse}-{day}')
reads_raw.columns = col_names
for j in to_drop:
    reads_raw.drop(j, axis=1, inplace=True)
# reads_raw.to_csv('mcnulty-datasets/reads.tsv', sep='\t')
reads_raw

,01-00,01-01,01-02,01-04,01-06,01-09,01-11,01-12,01-14,01-15,...,15-25,15-26,15-28,15-29,15-30,15-32,15-34,15-37,15-39,15-40
name,,,,,,,,,,,,,,,,,,,,,
B_caccae,28518,72598,90757,90761,43961,38713,47831,42591,78010,108215,...,121543,119269,27031,73923,65570,203081,204656,225795,261540,145981
B_cellulosilyticus_WH2,61078,281681,436720,834727,312998,49724,229893,311218,329145,151393,...,665898,755282,55939,75706,57077,256189,224229,313596,246003,75657
B_ovatus,66063,169916,157855,182563,69461,5558,77031,86669,80831,33503,...,260993,294711,17201,7579,1767,5301,6507,6843,8071,6086
B_thetaiotaomicron,24562,76142,147517,211588,82272,14337,54122,55977,59906,73198,...,101532,90300,23595,53246,42963,137392,118051,164264,204214,183137
B_uniformis,4428,4617,14112,32379,14689,3338,15498,15371,19033,14780,...,47209,56028,4612,11342,7656,22550,17774,27056,23763,18286
B_vulgatus,412906,132224,104042,159878,77457,28659,36031,39085,87799,79556,...,193970,230247,53179,91247,44173,167332,200621,232121,328177,161095
C_aerofaciens,187,2844,5418,6777,3186,205,2419,2415,1190,3552,...,1928,2070,726,1062,711,347,668,536,358,579
C_scindens,15886,11403,10153,11103,5237,1321,6789,4148,2489,8034,...,10387,11376,5187,6106,7428,6985,11135,7155,7066,16116
C_spiroforme,76,1664,3604,1482,159,314,183,33,60,57,...,146,133,48,50,54,380,660,292,405,481


### Norm reads

In [5]:
reads_norm = pd.read_excel('McNulty_data_source.xlsx', sheet_name='Norm_counts')
reads_norm

,name,01-01,01-01-r1,01-02,01-03,01-05,01-07,01-10,01-12,01-13,...,15-29,15-30,15-31,15-33,15-35,15-38,15-40,15-41,15-42,15-CEC
0,B_caccae_ATCC_43185,1075.9,6581.3,16753.9,20944.5,20945.4,10145.1,8934.0,11038.2,9829.0,...,6238.1,17059.6,15132.0,46866.2,47229.6,52108.0,60357.1,33688.9,57836.1,92666.1
1,B_ovatus_ATCC_8483,1355.7,10751.7,27653.7,25690.8,29712.0,11304.7,904.6,12536.7,14105.3,...,2799.4,1233.5,287.6,862.7,1059.0,1113.7,1313.5,990.5,2468.1,2757.9
2,B_thetaiotaomicron_VPI-5482,615.1,4151.9,12870.7,24935.7,35765.9,13906.9,2423.5,9148.6,9462.1,...,3988.4,9000.5,7262.3,23224.2,19954.8,27766.5,34519.5,30956.7,42735.6,56138.5
3,B_uniformis_ATCC_8492,101.7,1023.3,1066.9,3261.1,7482.4,3394.4,771.4,3581.4,3552.0,...,1065.8,2621.0,1769.2,5211.0,4107.3,6252.3,5491.3,4225.7,11461.7,12332.9
4,B_vulgatus_ATCC_8482,3736.6,87112.3,27895.8,21950.1,33730.1,16341.4,6046.3,7601.6,8245.9,...,11219.4,19250.7,9319.3,35302.7,42325.8,48971.4,69236.7,33986.8,56665.0,68868.6
5,B_WH2_closed,1282.5,8823.2,40690.9,63087.5,120582.5,45214.9,7183.0,33209.8,44957.8,...,8080.8,10936.3,8245.2,37008.4,32391.5,45301.3,35537.0,10929.2,12759.2,14592.7
6,C_aerofaciens_ATCC_25986,26.1,81.3,1237.0,2356.6,2947.7,1385.8,89.2,1052.2,1050.4,...,315.8,461.9,309.3,150.9,290.5,233.1,155.7,251.8,931.7,919.9
7,C_scindens_ATCC_35704,452.7,4595.3,3298.5,2936.9,3211.7,1514.9,382.1,1963.8,1199.9,...,1500.4,1766.2,2148.7,2020.5,3221.0,2069.7,2043.9,4661.8,10082.9,11451.1
8,C_spiroforme_DSM_1552,0.4,32.9,720.1,1559.6,641.3,68.8,135.9,79.2,14.3,...,20.8,21.6,23.4,164.4,285.6,126.4,175.3,208.2,191.3,608.9
9,D_longicatena_DSM_13814,13.5,275.9,38.4,2.5,0.4,0.7,0.0,0.4,0.4,...,2.5,3.6,3.6,7.1,10.3,10.7,17.4,5.0,7.1,16.7


In [6]:
reads_norm['name'].replace(to_replace, inplace=True)
reads_norm.sort_values(by='name', inplace=True)
reads_norm.set_index('name', inplace=True)
r1 = re.compile('\d\d-\d\d-r\d')
r1s = [col for col in reads_norm if r1.match(col)]
rs = [i.replace('-r1', '') for i in r1s]
for i,j in zip(r1s, rs):
    reads_norm[j] = reads_norm[i]
    reads_norm.drop(i, axis=1, inplace=True)
cec = re.compile('\d\d-CEC')
for col in reads_norm:
    if cec.match(col):
        reads_norm.drop(col, axis=1, inplace=True)
col_names = []
for col in reads_norm.columns:
    mouse, day = col.split('-')
    day = str(int(day) - 1)
    if len(day) == 1:
        day = '0' + day
    col_names.append(f'{mouse}-{day}')
reads_norm.columns = col_names
for j in to_drop:
    reads_norm.drop(j, axis=1, inplace=True)
# reads_norm.to_csv('mcnulty-datasets/reads.tsv', sep='\t')
reads_norm

,01-00,01-01,01-02,01-04,01-06,01-09,01-11,01-12,01-14,01-15,...,15-25,15-26,15-28,15-29,15-30,15-32,15-34,15-37,15-39,15-40
name,,,,,,,,,,,,,,,,,,,,,
B_caccae,6581.3,16753.9,20944.5,20945.4,10145.1,8934.0,11038.2,9829.0,18002.8,24973.4,...,28049.2,27524.4,6238.1,17059.6,15132.0,46866.2,47229.6,52108.0,60357.1,33688.9
B_cellulosilyticus_WH2,8823.2,40690.9,63087.5,120582.5,45214.9,7183.0,33209.8,44957.8,47547.4,21869.8,...,96193.9,109106.1,8080.8,10936.3,8245.2,37008.4,32391.5,45301.3,35537.0,10929.2
B_ovatus,10751.7,27653.7,25690.8,29712.0,11304.7,904.6,12536.7,14105.3,13155.2,5452.6,...,42476.4,47964.0,2799.4,1233.5,287.6,862.7,1059.0,1113.7,1313.5,990.5
B_thetaiotaomicron,4151.9,12870.7,24935.7,35765.9,13906.9,2423.5,9148.6,9462.1,10126.3,12373.1,...,17162.5,15263.9,3988.4,9000.5,7262.3,23224.2,19954.8,27766.5,34519.5,30956.7
B_uniformis,1023.3,1066.9,3261.1,7482.4,3394.4,771.4,3581.4,3552.0,4398.3,3415.5,...,10909.4,12947.4,1065.8,2621.0,1769.2,5211.0,4107.3,6252.3,5491.3,4225.7
B_vulgatus,87112.3,27895.8,21950.1,33730.1,16341.4,6046.3,7601.6,8245.9,18523.3,16784.2,...,40922.6,48576.1,11219.4,19250.7,9319.3,35302.7,42325.8,48971.4,69236.7,33986.8
C_aerofaciens,81.3,1237.0,2356.6,2947.7,1385.8,89.2,1052.2,1050.4,517.6,1545.0,...,838.6,900.4,315.8,461.9,309.3,150.9,290.5,233.1,155.7,251.8
C_scindens,4595.3,3298.5,2936.9,3211.7,1514.9,382.1,1963.8,1199.9,720.0,2324.0,...,3004.6,3290.7,1500.4,1766.2,2148.7,2020.5,3221.0,2069.7,2043.9,4661.8
C_spiroforme,32.9,720.1,1559.6,641.3,68.8,135.9,79.2,14.3,26.0,24.7,...,63.2,57.6,20.8,21.6,23.4,164.4,285.6,126.4,175.3,208.2


## Metadata

In [7]:
metadata = reads_norm.T.copy()
metadata['sampleID']=metadata.index
metadata = metadata.loc[:, 'sampleID'].str.split('-', expand=True)
metadata.reset_index(inplace=True)
metadata.columns = ['sampleID', 'subject', 'time']
metadata.set_index('sampleID', inplace=True)
# metadata = metadata[metadata['time'] != 'CEC']
metadata['time'] = metadata['time'].astype(float)
metadata['subject'] = metadata['subject'].astype(int)
# metadata.to_csv('mcnulty-datasets/metadata.tsv', sep='\t')
metadata

,subject,time
sampleID,,
01-00,1,0.0
01-01,1,1.0
01-02,1,2.0
01-04,1,4.0
01-06,1,6.0
...,...,...
15-32,15,32.0
15-34,15,34.0
15-37,15,37.0


## DNA yields (proxy to qPCR)

In [8]:
# dna_yields = pd.read_excel('McNulty_data_source.xlsx', sheet_name='DNA_yields')
dna_yields = pd.read_excel('McNulty_data_source.xlsx', sheet_name='DNA_per_sample')
# dna_yields['Day'] = dna_yields['Day'].replace('cecum', 'CEC')
dna_yields = dna_yields[dna_yields['Day'] != 'cecum']
dna_yields

,Mouse,Day,ug DNA / mg starting material
0,1,1,0.56
1,2,1,0.66
2,3,1,0.50
3,4,1,0.23
4,5,1,0.20
...,...,...,...
370,11,42,0.57
371,12,42,0.63
372,13,42,0.43
373,14,42,0.59


In [9]:
N_A = 6.02214076e23 # Avogadro's number
dsDNA_bp = 660 # Average mass of a DNA bp (g/mol)
unit_conversion = 1e6 # Convert from ug to g
conversion_factor = N_A/(dsDNA_bp*unit_conversion)
conversion_factor

912445569696969.6

In [10]:
genome_size = pd.read_excel('McNulty_data_source.xlsx', sheet_name='Genome_size')
genome_size

,name,Genome_length (bp)
0,B_caccae,4564814
1,B_cellulosilyticus_WH2,7082128
2,B_ovatus,6463169
3,B_thetaiotaomicron,6293399
4,B_uniformis,4717497
5,B_vulgatus,5163189
6,C_aerofaciens,2439869
7,C_scindens,3619444
8,C_spiroforme,2507485
9,D_longicatena,2913833


In [11]:
reads_raw.reset_index(inplace=True)
raw_counts_long = reads_raw.melt(id_vars=['name'], var_name='Mouse-Day', value_name='Raw_counts')
raw_counts_long['Mouse'] = raw_counts_long['Mouse-Day'].str.split('-').str[0]
raw_counts_long['Day'] = raw_counts_long['Mouse-Day'].str.split('-').str[1]
raw_counts_long = raw_counts_long[raw_counts_long['Day'] != 'CEC']
raw_counts_long['Mouse'] = raw_counts_long['Mouse'].astype(int)
raw_counts_long['Day'] = raw_counts_long['Day'].astype(int)
raw_counts_long.drop('Mouse-Day', axis=1, inplace=True)
raw_counts_long = pd.merge(raw_counts_long, genome_size, on='name')
raw_counts_long

,name,Raw_counts,Mouse,Day,Genome_length (bp)
0,B_caccae,28518,1,0,4564814
1,B_caccae,72598,1,1,4564814
2,B_caccae,90757,1,2,4564814
3,B_caccae,90761,1,4,4564814
4,B_caccae,43961,1,6,4564814
...,...,...,...,...,...
4303,R_obeum,3011,15,32,3624307
4304,R_obeum,4078,15,34,3624307
4305,R_obeum,2730,15,37,3624307
4306,R_obeum,3453,15,39,3624307


In [12]:
# dna_yields['Day'].replace('cecum', 'CEC', inplace=True)
# dna_yields_clean = dna_yields[dna_yields['Day'] != 'CEC'].copy()
dna_yields['Day'] = dna_yields['Day'].astype(int) - 1
dna_yields['Mouse'] = dna_yields['Mouse'].astype(int)
dna_yields

,Mouse,Day,ug DNA / mg starting material
0,1,0,0.56
1,2,0,0.66
2,3,0,0.50
3,4,0,0.23
4,5,0,0.20
...,...,...,...
370,11,41,0.57
371,12,41,0.63
372,13,41,0.43
373,14,41,0.59


In [13]:
full_table = pd.merge(raw_counts_long, dna_yields, on=['Mouse', 'Day'])
full_table = full_table[full_table.columns[[0,4,2,3,1,5]]].copy()
full_table

,name,Genome_length (bp),Mouse,Day,Raw_counts,ug DNA / mg starting material
0,B_caccae,4564814,1,0,28518,0.56
1,B_cellulosilyticus_WH2,7082128,1,0,61078,0.56
2,B_ovatus,6463169,1,0,66063,0.56
3,B_thetaiotaomicron,6293399,1,0,24562,0.56
4,B_uniformis,4717497,1,0,4428,0.56
...,...,...,...,...,...,...
4303,C_scindens,3619444,15,40,16116,0.72
4304,C_spiroforme,2507485,15,40,481,0.72
4305,D_longicatena,2913833,15,40,14,0.72
4306,P_distasonis,4811379,15,40,21768,0.72


In [14]:
total_counts = full_table.groupby(['Mouse', 'Day']).sum().reset_index().rename(columns={'Raw_counts':'Total_raw_counts'}).iloc[:,[0,1,3]].copy()
total_counts

,Mouse,Day,Total_raw_counts
0,1,0,616386
1,1,1,780273
2,1,2,1039638
3,1,4,1642210
4,1,6,661011
...,...,...,...
354,15,32,819196
355,15,34,806001
356,15,37,1007378
357,15,39,1113621


In [15]:
full_table = pd.merge(full_table, total_counts, on=['Mouse', 'Day'])
full_table['Relative_counts'] = full_table['Raw_counts']/full_table['Total_raw_counts']
# full_table['measurement1'] = ((full_table['Relative_counts']*full_table['DNA Yield (ug)'])/full_table['Genome_length (bp)'])*conversion_factor
full_table['measurement1'] = ((full_table['Relative_counts']*full_table['ug DNA / mg starting material'])/full_table['Genome_length (bp)'])*conversion_factor
to_repl = {1:'LF0', 2:'LF0', 3:'LF0', 4:'LF0', 5:'LF0', 6:'LF0', 7:'LF0', 8:'HF0', 9:'HF0', 10:'HF0', 11:'HF0', 12:'HF0', 13:'HF0', 14:'HF0', 15:'HF0'}
full_table['Treatment'] = full_table['Mouse'].replace(to_repl)
full_table = full_table[full_table.columns[[0,1,9,2,3,4,5,7,6,8]]].copy()
full_table

,name,Genome_length (bp),Treatment,Mouse,Day,Raw_counts,ug DNA / mg starting material,Relative_counts,Total_raw_counts,measurement1
0,B_caccae,4564814,LF0,1,0,28518,0.56,0.046266,616386,5.178908e+06
1,B_cellulosilyticus_WH2,7082128,LF0,1,0,61078,0.56,0.099091,616386,7.149296e+06
2,B_ovatus,6463169,LF0,1,0,66063,0.56,0.107178,616386,8.473347e+06
3,B_thetaiotaomicron,6293399,LF0,1,0,24562,0.56,0.039848,616386,3.235346e+06
4,B_uniformis,4717497,LF0,1,0,4428,0.56,0.007184,616386,7.781050e+05
...,...,...,...,...,...,...,...,...,...,...
4303,C_scindens,3619444,HF0,15,40,16116,0.72,0.025183,639944,4.571018e+06
4304,C_spiroforme,2507485,HF0,15,40,481,0.72,0.000752,639944,1.969265e+05
4305,D_longicatena,2913833,HF0,15,40,14,0.72,0.000022,639944,4.932429e+03
4306,P_distasonis,4811379,HF0,15,40,21768,0.72,0.034015,639944,4.644580e+06


In [16]:
table = full_table.loc[:,['name', 'Treatment', 'Mouse', 'Day', 'measurement1']].copy()
table

,name,Treatment,Mouse,Day,measurement1
0,B_caccae,LF0,1,0,5.178908e+06
1,B_cellulosilyticus_WH2,LF0,1,0,7.149296e+06
2,B_ovatus,LF0,1,0,8.473347e+06
3,B_thetaiotaomicron,LF0,1,0,3.235346e+06
4,B_uniformis,LF0,1,0,7.781050e+05
...,...,...,...,...,...
4303,C_scindens,HF0,15,40,4.571018e+06
4304,C_spiroforme,HF0,15,40,1.969265e+05
4305,D_longicatena,HF0,15,40,4.932429e+03
4306,P_distasonis,HF0,15,40,4.644580e+06


In [17]:
table_summary = table.groupby(['Mouse', 'Day']).sum(). \
    sort_values(by=['Mouse', 'Day']). \
    reset_index(). \
    copy()
table_summary

,Mouse,Day,measurement1
0,1,0,9.545881e+07
1,1,1,1.446444e+08
2,1,2,3.673888e+08
3,1,4,2.668750e+08
4,1,6,2.934825e+08
...,...,...,...
354,15,32,4.044633e+08
355,15,34,3.334795e+08
356,15,37,3.641306e+08
357,15,39,3.658736e+08


In [18]:
dna_yields = table_summary.copy()
dna_yields.sort_values(by=['Mouse', 'Day'], inplace=True)
dna_yields['sampleID'] = dna_yields['Mouse'].astype(str) + '-' + dna_yields['Day'].astype(str)
to_rename = {'^1-':'01-', '^2-':'02-', '^3-':'03-', '^4-':'04-', '^5-':'05-', '^6-':'06-', '^7-':'07-', '^8-':'08-', '^9-':'09-',
              '-0':'-00','-1$':'-01', '-2$':'-02', '-3$':'-03', '-4$':'-04', '-5$':'-05', '-6$':'-06', '-7$':'-07', '-8$':'-08', '-9$':'-09'}
dna_yields['sampleID'] = dna_yields['sampleID'].replace(to_replace=to_rename, regex=True)
dna_yields = dna_yields.loc[:,['sampleID', 'measurement1']]
dna_yields.set_index('sampleID', inplace=True)
dna_yields.columns = ['measurement1']
np.random.seed(0)
def add_noise(x, eta=0.05):
    return np.random.normal(loc=x, scale=eta*x)
dna_yields['measurement2'] = dna_yields['measurement1'].apply(add_noise)
dna_yields['measurement3'] = dna_yields['measurement1'].apply(add_noise)  
# dna_yields.to_csv('mcnulty-datasets/dna_yields.tsv', sep='\t')
dna_yields

,measurement1,measurement2,measurement3
sampleID,,,
01-00,9.545881e+07,1.038785e+08,9.841974e+07
01-01,1.446444e+08,1.475384e+08,1.496958e+08
01-02,3.673888e+08,3.853677e+08,3.674581e+08
01-04,2.668750e+08,2.967769e+08,2.793094e+08
01-06,2.934825e+08,3.208873e+08,2.984712e+08
...,...,...,...
15-32,4.044633e+08,3.912719e+08,3.985906e+08
15-34,3.334795e+08,3.247892e+08,3.556189e+08
15-37,3.641306e+08,3.305747e+08,3.622866e+08


## Perturbations

In [19]:
perturbations = pd.read_excel('McNulty_data_source.xlsx', sheet_name='Perturbations', index_col=0)
perturbations['start'] = perturbations['start'] - 1
perturbations['end'] = perturbations['end'] - 1
# perturbations.to_csv('mcnulty-datasets/perturbations.tsv', sep='\t')
perturbations

,start,end,subject
name,,,
HF/HS,14,26,1
HF/HS,14,26,2
HF/HS,14,26,3
HF/HS,14,26,4
HF/HS,14,26,5
HF/HS,14,26,6
HF/HS,14,26,7
LF/HPP,14,26,8
LF/HPP,14,26,9


## Taxonomy

In [20]:
taxonomy = pd.read_excel('McNulty_data_source.xlsx', sheet_name='Taxonomy', index_col=0)
# taxonomy.to_csv('mcnulty-datasets/taxonomy.tsv', sep='\t')
taxonomy

,sequence,kingdom,phylum,class,order,family,genus,species
name,,,,,,,,
B_caccae,GACAGTATATCTTATCTTTATGATGCTGCGGGCAGGAAACTCCGTA...,Bacteria,Bacteroidetes,Bacteroidia,Bacteroidales,Bacteroidaceae,Bacteroides,caccae
B_cellulosilyticus_WH2,GAACTTTCCGATTTACCGACCCACATGCCAAGGACTTCTTTCAAAC...,Bacteria,Bacteroidetes,Bacteroidia,Bacteroidales,Bacteroidaceae,Bacteroides,cellulosilyticus
B_ovatus,TGTAAATGACCGTAAGGCACAGCATGAAGGCCGATGGTGTGCAGGT...,Bacteria,Bacteroidetes,Bacteroidia,Bacteroidales,Bacteroidaceae,Bacteroides,ovatus
B_thetaiotaomicron,TAGGTGCAGGTACAACTTTTGACCACTGTCAGGCATACTACGGTGA...,Bacteria,Bacteroidetes,Bacteroidia,Bacteroidales,Bacteroidaceae,Bacteroides,thetaiotaomicron
B_uniformis,TCGTACTTGGTACGTTTGGTATGTAAAATCACCATCTACACCGGTA...,Bacteria,Bacteroidetes,Bacteroidia,Bacteroidales,Bacteroidaceae,Bacteroides,uniformis
B_vulgatus,ATGTTTCATATGATTGAAAACGATCACGTCGTTTTATGGGGTCGTT...,Bacteria,Bacteroidetes,Bacteroidia,Bacteroidales,Bacteroidaceae,Phocaeicola,vulgatus
C_aerofaciens,TTTCTATTTGCCATTCATACCACCTAGTCTCGTTTAAACAGGTCGC...,Bacteria,Actinobacteria,Coriobacteriia,Coriobacteriales,Coriobacteriaceae,Collinsella,aerofaciens
C_scindens,ATAAAATGGATATCATTCGCAAGTCAGCCCTTAGTCATTCATAGTC...,Bacteria,Firmicutes,Clostridia,Eubacteriales,Lachnospiraceae,Lachnoclostridium,scindens
C_spiroforme,CTGGAGCTGCTGAGGCTACAGTTATATCACAGTATGTATCTGGTAT...,Bacteria,Firmicutes,Erysipelotrichia,Erysipelotrichales,Erysipelotrichaceae,Erysipelatoclostridium,spiroforme


# Data segregation by treatment

## LF/HPP ==> HF/HS ==> LF/HPP

### Reads

In [21]:
lf0 = re.compile('0[1,2,3,4,5,6,7]-\d\d')
lf0s = [col for col in reads_raw if lf0.match(col)]

In [22]:
reads_raw_lf0 = reads_raw.loc[:,lf0s]
# reads_raw_lf0.to_csv('mcnulty-datasets/LF0/reads.tsv', sep='\t')
reads_raw_lf0

,01-00,01-01,01-02,01-04,01-06,01-09,01-11,01-12,01-14,01-15,...,07-25,07-26,07-28,07-29,07-30,07-32,07-34,07-37,07-39,07-40
0,28518,72598,90757,90761,43961,38713,47831,42591,78010,108215,...,418165,132064,88301,13732,24222,41296,57435,29996,2895,4626
1,61078,281681,436720,834727,312998,49724,229893,311218,329145,151393,...,406839,132306,211381,83236,143535,380120,317512,264989,265127,370758
2,66063,169916,157855,182563,69461,5558,77031,86669,80831,33503,...,18628,7706,28410,8833,27814,58150,131016,250016,74337,115868
3,24562,76142,147517,211588,82272,14337,54122,55977,59906,73198,...,244343,91532,159599,14399,27582,51837,99981,99616,30796,61547
4,4428,4617,14112,32379,14689,3338,15498,15371,19033,14780,...,97667,29714,28921,2738,3270,15448,52085,45160,6522,13939
5,412906,132224,104042,159878,77457,28659,36031,39085,87799,79556,...,289457,97164,75963,30433,36118,78786,173592,104361,75429,97616
6,187,2844,5418,6777,3186,205,2419,2415,1190,3552,...,1297,636,1397,31,233,267,800,864,110,485
7,15886,11403,10153,11103,5237,1321,6789,4148,2489,8034,...,23917,7179,14720,352,2251,2322,5392,4642,801,3434
8,76,1664,3604,1482,159,314,183,33,60,57,...,1087,96,173,5,45,131,327,406,34,174
9,775,108,7,1,2,0,1,1,0,0,...,29,13,45,2,11,13,44,33,36,66


In [23]:
reads_norm_lf0 = reads_norm.loc[:,lf0s]
reads_norm_lf0.to_csv('mcnulty-datasets/LF0/reads.tsv', sep='\t')
reads_norm_lf0

,01-00,01-01,01-02,01-04,01-06,01-09,01-11,01-12,01-14,01-15,...,07-25,07-26,07-28,07-29,07-30,07-32,07-34,07-37,07-39,07-40
name,,,,,,,,,,,,,,,,,,,,,
B_caccae,6581.3,16753.9,20944.5,20945.4,10145.1,8934.0,11038.2,9829.0,18002.8,24973.4,...,96502.3,30477.2,20377.7,3169.0,5589.8,9530.1,13254.6,6922.3,668.1,1067.6
B_cellulosilyticus_WH2,8823.2,40690.9,63087.5,120582.5,45214.9,7183.0,33209.8,44957.8,47547.4,21869.8,...,58770.9,19112.6,30535.6,12024.1,20734.7,54911.2,45867.0,38279.6,38299.6,53558.7
B_ovatus,10751.7,27653.7,25690.8,29712.0,11304.7,904.6,12536.7,14105.3,13155.2,5452.6,...,3031.7,1254.1,4623.7,1437.6,4526.7,9463.9,21322.8,40689.9,12098.3,18857.4
B_thetaiotaomicron,4151.9,12870.7,24935.7,35765.9,13906.9,2423.5,9148.6,9462.1,10126.3,12373.1,...,41302.7,15472.2,26977.9,2433.9,4662.3,8762.3,16900.4,16838.7,5205.6,10403.6
B_uniformis,1023.3,1066.9,3261.1,7482.4,3394.4,771.4,3581.4,3552.0,4398.3,3415.5,...,22569.6,6866.5,6683.3,632.7,755.7,3569.8,12036.2,10435.9,1507.2,3221.1
B_vulgatus,87112.3,27895.8,21950.1,33730.1,16341.4,6046.3,7601.6,8245.9,18523.3,16784.2,...,61067.8,20499.1,16026.2,6420.6,7619.9,16621.8,36623.4,22017.4,15913.5,20594.4
C_aerofaciens,81.3,1237.0,2356.6,2947.7,1385.8,89.2,1052.2,1050.4,517.6,1545.0,...,564.1,276.6,607.6,13.5,101.3,116.1,348.0,375.8,47.8,211.0
C_scindens,4595.3,3298.5,2936.9,3211.7,1514.9,382.1,1963.8,1199.9,720.0,2324.0,...,6918.3,2076.6,4258.0,101.8,651.1,671.7,1559.7,1342.8,231.7,993.3
C_spiroforme,32.9,720.1,1559.6,641.3,68.8,135.9,79.2,14.3,26.0,24.7,...,470.4,41.5,74.9,2.2,19.5,56.7,141.5,175.7,14.7,75.3


### Metadata

In [24]:
metadata_lf0 = metadata.loc[lf0s]
metadata_lf0.to_csv('mcnulty-datasets/LF0/metadata.tsv', sep='\t')
metadata_lf0

,subject,time
sampleID,,
01-00,1,0.0
01-01,1,1.0
01-02,1,2.0
01-04,1,4.0
01-06,1,6.0
...,...,...
07-32,7,32.0
07-34,7,34.0
07-37,7,37.0


### DNA Yields

In [25]:
dna_yields_lf0 = dna_yields.loc[lf0s]
dna_yields_lf0.to_csv('mcnulty-datasets/LF0/dna_yields.tsv', sep='\t')
dna_yields_lf0

,measurement1,measurement2,measurement3
sampleID,,,
01-00,9.545881e+07,1.038785e+08,9.841974e+07
01-01,1.446444e+08,1.475384e+08,1.496958e+08
01-02,3.673888e+08,3.853677e+08,3.674581e+08
01-04,2.668750e+08,2.967769e+08,2.793094e+08
01-06,2.934825e+08,3.208873e+08,2.984712e+08
...,...,...,...
07-32,3.680686e+08,3.696799e+08,3.371284e+08
07-34,4.610941e+08,4.328421e+08,4.405992e+08
07-37,2.957601e+08,3.082465e+08,2.993405e+08


### Pertrubations

In [26]:
perturbations_lf0 = perturbations[perturbations.index == 'HF/HS']
perturbations_lf0.to_csv('mcnulty-datasets/LF0/perturbations.tsv', sep='\t')
perturbations_lf0

,start,end,subject
name,,,
HF/HS,14,26,1
HF/HS,14,26,2
HF/HS,14,26,3
HF/HS,14,26,4
HF/HS,14,26,5
HF/HS,14,26,6
HF/HS,14,26,7


### Taxonomy

In [37]:
taxonomy.to_csv('mcnulty-datasets/LF0/taxonomy.tsv', sep='\t')
taxonomy

,sequence,kingdom,phylum,class,order,family,genus,species
name,,,,,,,,
B_caccae,GACAGTATATCTTATCTTTATGATGCTGCGGGCAGGAAACTCCGTA...,Bacteria,Bacteroidetes,Bacteroidia,Bacteroidales,Bacteroidaceae,Bacteroides,caccae
B_cellulosilyticus_WH2,GAACTTTCCGATTTACCGACCCACATGCCAAGGACTTCTTTCAAAC...,Bacteria,Bacteroidetes,Bacteroidia,Bacteroidales,Bacteroidaceae,Bacteroides,cellulosilyticus
B_ovatus,TGTAAATGACCGTAAGGCACAGCATGAAGGCCGATGGTGTGCAGGT...,Bacteria,Bacteroidetes,Bacteroidia,Bacteroidales,Bacteroidaceae,Bacteroides,ovatus
B_thetaiotaomicron,TAGGTGCAGGTACAACTTTTGACCACTGTCAGGCATACTACGGTGA...,Bacteria,Bacteroidetes,Bacteroidia,Bacteroidales,Bacteroidaceae,Bacteroides,thetaiotaomicron
B_uniformis,TCGTACTTGGTACGTTTGGTATGTAAAATCACCATCTACACCGGTA...,Bacteria,Bacteroidetes,Bacteroidia,Bacteroidales,Bacteroidaceae,Bacteroides,uniformis
B_vulgatus,ATGTTTCATATGATTGAAAACGATCACGTCGTTTTATGGGGTCGTT...,Bacteria,Bacteroidetes,Bacteroidia,Bacteroidales,Bacteroidaceae,Phocaeicola,vulgatus
C_aerofaciens,TTTCTATTTGCCATTCATACCACCTAGTCTCGTTTAAACAGGTCGC...,Bacteria,Actinobacteria,Coriobacteriia,Coriobacteriales,Coriobacteriaceae,Collinsella,aerofaciens
C_scindens,ATAAAATGGATATCATTCGCAAGTCAGCCCTTAGTCATTCATAGTC...,Bacteria,Firmicutes,Clostridia,Eubacteriales,Lachnospiraceae,Lachnoclostridium,scindens
C_spiroforme,CTGGAGCTGCTGAGGCTACAGTTATATCACAGTATGTATCTGGTAT...,Bacteria,Firmicutes,Erysipelotrichia,Erysipelotrichales,Erysipelotrichaceae,Erysipelatoclostridium,spiroforme


##  HF/HS ==> LF/HPP ==> HF/HS

### Reads

In [28]:
hf0 = re.compile('(0[8,9]|1[0,1,2,3,4,5])-\d\d')
hf0s = [col for col in reads_raw if hf0.match(col)]

In [29]:
reads_raw_hf0 = reads_raw.loc[:,hf0s]
# reads_raw_hf0.to_csv('mcnulty-datasets/HF0/reads.tsv', sep='\t')
reads_raw_hf0

,08-00,08-01,08-02,08-04,08-06,08-09,08-11,08-12,08-14,08-15,...,15-25,15-26,15-28,15-29,15-30,15-32,15-34,15-37,15-39,15-40
0,123485,217557,731821,545096,467105,361281,193626,162890,101238,152392,...,121543,119269,27031,73923,65570,203081,204656,225795,261540,145981
1,244631,107702,304392,329165,343676,320709,167910,133854,277134,354858,...,665898,755282,55939,75706,57077,256189,224229,313596,246003,75657
2,278571,69850,58913,14421,28221,22873,16324,12494,22709,80175,...,260993,294711,17201,7579,1767,5301,6507,6843,8071,6086
3,62222,131370,498206,94726,90837,119823,137859,93266,104824,214438,...,101532,90300,23595,53246,42963,137392,118051,164264,204214,183137
4,3556,2914,36157,57117,39156,31142,19514,15289,19361,26007,...,47209,56028,4612,11342,7656,22550,17774,27056,23763,18286
5,184603,143753,261522,234137,437758,326971,238217,155188,110953,110320,...,193970,230247,53179,91247,44173,167332,200621,232121,328177,161095
6,1564,1488,5113,2212,3050,718,868,1132,1155,2978,...,1928,2070,726,1062,711,347,668,536,358,579
7,13853,14305,68457,21965,46540,12403,7303,6770,8072,20041,...,10387,11376,5187,6106,7428,6985,11135,7155,7066,16116
8,2880,1116,8225,5493,778,154,182,127,173,1837,...,146,133,48,50,54,380,660,292,405,481
9,382,31,7,1,8,0,2,1,1,1,...,65,115,7,10,10,20,29,30,49,14


In [30]:
reads_norm_hf0 = reads_norm.loc[:,hf0s]
reads_norm_hf0.to_csv('mcnulty-datasets/HF0/reads.tsv', sep='\t')
reads_norm_hf0

,08-00,08-01,08-02,08-04,08-06,08-09,08-11,08-12,08-14,08-15,...,15-25,15-26,15-28,15-29,15-30,15-32,15-34,15-37,15-39,15-40
name,,,,,,,,,,,,,,,,,,,,,
B_caccae,28497.3,50206.9,168886.5,125794.9,107796.5,83374.9,44684.2,37591.0,23363.3,35168.4,...,28049.2,27524.4,6238.1,17059.6,15132.0,46866.2,47229.6,52108.0,60357.1,33688.9
B_cellulosilyticus_WH2,35338.8,15558.4,43971.7,47550.3,49646.6,46328.8,24255.8,19336.2,40034.1,51261.9,...,96193.9,109106.1,8080.8,10936.3,8245.2,37008.4,32391.5,45301.3,35537.0,10929.2
B_ovatus,45337.2,11368.0,9588.0,2347.0,4592.9,3722.6,2656.7,2033.4,3695.9,13048.4,...,42476.4,47964.0,2799.4,1233.5,287.6,862.7,1059.0,1113.7,1313.5,990.5
B_thetaiotaomicron,10517.7,22206.2,84214.7,16012.1,15354.7,20254.4,23303.1,15765.3,17719.0,36247.7,...,17162.5,15263.9,3988.4,9000.5,7262.3,23224.2,19954.8,27766.5,34519.5,30956.7
B_uniformis,821.7,673.4,8355.4,13199.0,9048.5,7196.5,4509.4,3533.1,4474.1,6009.9,...,10909.4,12947.4,1065.8,2621.0,1769.2,5211.0,4107.3,6252.3,5491.3,4225.7
B_vulgatus,38946.4,30328.1,55174.3,49396.8,92355.4,68982.3,50257.5,32740.6,23408.2,23274.6,...,40922.6,48576.1,11219.4,19250.7,9319.3,35302.7,42325.8,48971.4,69236.7,33986.8
C_aerofaciens,680.3,647.2,2223.9,962.1,1326.6,312.3,377.5,492.4,502.4,1295.3,...,838.6,900.4,315.8,461.9,309.3,150.9,290.5,233.1,155.7,251.8
C_scindens,4007.2,4137.9,19802.2,6353.7,13462.4,3587.7,2112.5,1958.3,2334.9,5797.2,...,3004.6,3290.7,1500.4,1766.2,2148.7,2020.5,3221.0,2069.7,2043.9,4661.8
C_spiroforme,1246.3,483.0,3559.4,2377.1,336.7,66.6,78.8,55.0,74.9,795.0,...,63.2,57.6,20.8,21.6,23.4,164.4,285.6,126.4,175.3,208.2


### Metadata

In [31]:
metadata_hf0 = metadata.loc[hf0s]
metadata_hf0.to_csv('mcnulty-datasets/HF0/metadata.tsv', sep='\t')
metadata_hf0

,subject,time
sampleID,,
08-00,8,0.0
08-01,8,1.0
08-02,8,2.0
08-04,8,4.0
08-06,8,6.0
...,...,...
15-32,15,32.0
15-34,15,34.0
15-37,15,37.0


### DNA Yields

In [32]:
dna_yields_hf0 = dna_yields.loc[hf0s]
dna_yields_hf0.to_csv('mcnulty-datasets/HF0/dna_yields.tsv', sep='\t')
dna_yields_hf0

,measurement1,measurement2,measurement3
sampleID,,,
08-00,1.349494e+08,1.429656e+08,1.444790e+08
08-01,3.068263e+08,3.116886e+08,2.704737e+08
08-02,2.187238e+08,2.287945e+08,2.281733e+08
08-04,8.075566e+07,8.204261e+07,7.171262e+07
08-06,8.563187e+07,8.930047e+07,8.735093e+07
...,...,...,...
15-32,4.044633e+08,3.912719e+08,3.985906e+08
15-34,3.334795e+08,3.247892e+08,3.556189e+08
15-37,3.641306e+08,3.305747e+08,3.622866e+08


### Pertrubations

In [33]:
perturbations_hf0 = perturbations[perturbations.index == 'LF/HPP']
perturbations_hf0.to_csv('mcnulty-datasets/HF0/perturbations.tsv', sep='\t')
perturbations_hf0

,start,end,subject
name,,,
LF/HPP,14,26,8
LF/HPP,14,26,9
LF/HPP,14,26,10
LF/HPP,14,26,11
LF/HPP,14,26,12
LF/HPP,14,26,13
LF/HPP,14,26,14
LF/HPP,14,26,15


### Taxonomy

In [35]:
taxonomy.to_csv('mcnulty-datasets/HF0/taxonomy.tsv', sep='\t')
taxonomy

,sequence,kingdom,phylum,class,order,family,genus,species
name,,,,,,,,
B_caccae,GACAGTATATCTTATCTTTATGATGCTGCGGGCAGGAAACTCCGTA...,Bacteria,Bacteroidetes,Bacteroidia,Bacteroidales,Bacteroidaceae,Bacteroides,caccae
B_cellulosilyticus_WH2,GAACTTTCCGATTTACCGACCCACATGCCAAGGACTTCTTTCAAAC...,Bacteria,Bacteroidetes,Bacteroidia,Bacteroidales,Bacteroidaceae,Bacteroides,cellulosilyticus
B_ovatus,TGTAAATGACCGTAAGGCACAGCATGAAGGCCGATGGTGTGCAGGT...,Bacteria,Bacteroidetes,Bacteroidia,Bacteroidales,Bacteroidaceae,Bacteroides,ovatus
B_thetaiotaomicron,TAGGTGCAGGTACAACTTTTGACCACTGTCAGGCATACTACGGTGA...,Bacteria,Bacteroidetes,Bacteroidia,Bacteroidales,Bacteroidaceae,Bacteroides,thetaiotaomicron
B_uniformis,TCGTACTTGGTACGTTTGGTATGTAAAATCACCATCTACACCGGTA...,Bacteria,Bacteroidetes,Bacteroidia,Bacteroidales,Bacteroidaceae,Bacteroides,uniformis
B_vulgatus,ATGTTTCATATGATTGAAAACGATCACGTCGTTTTATGGGGTCGTT...,Bacteria,Bacteroidetes,Bacteroidia,Bacteroidales,Bacteroidaceae,Phocaeicola,vulgatus
C_aerofaciens,TTTCTATTTGCCATTCATACCACCTAGTCTCGTTTAAACAGGTCGC...,Bacteria,Actinobacteria,Coriobacteriia,Coriobacteriales,Coriobacteriaceae,Collinsella,aerofaciens
C_scindens,ATAAAATGGATATCATTCGCAAGTCAGCCCTTAGTCATTCATAGTC...,Bacteria,Firmicutes,Clostridia,Eubacteriales,Lachnospiraceae,Lachnoclostridium,scindens
C_spiroforme,CTGGAGCTGCTGAGGCTACAGTTATATCACAGTATGTATCTGGTAT...,Bacteria,Firmicutes,Erysipelotrichia,Erysipelotrichales,Erysipelotrichaceae,Erysipelatoclostridium,spiroforme
